In [8]:
import os

In [13]:
!kaggle datasets download tusharpadhy/deepfake-dataset

Dataset URL: https://www.kaggle.com/datasets/tusharpadhy/deepfake-dataset
License(s): CC0-1.0
deepfake-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
dataset = 'C:/Users/rutup/OneDrive/Desktop/robust-DL/project/kaggle'
os.makedirs(dataset, exist_ok=True)

!unzip deepfake-dataset.zip -d dataset

Archive:  deepfake-dataset.zip
  inflating: dataset/test/Fake/00276TOPP4.jpg  
  inflating: dataset/test/Fake/008BYSE725.jpg  
  inflating: dataset/test/Fake/009ZTJ3621.jpg  
  inflating: dataset/test/Fake/00F8LKY6JC.jpg  
  inflating: dataset/test/Fake/00JEP4Z36Z.jpg  
  inflating: dataset/test/Fake/00KEKJJ1Q4.jpg  
  inflating: dataset/test/Fake/00MZYXAT77.jpg  
  inflating: dataset/test/Fake/00PB1BNIE8.jpg  
  inflating: dataset/test/Fake/00QKZTHTLF.jpg  
  inflating: dataset/test/Fake/00V5CZZSSO.jpg  
  inflating: dataset/test/Fake/00XUQJZGHU.jpg  
  inflating: dataset/test/Fake/01050DBM3C.jpg  
  inflating: dataset/test/Fake/010MNNNOZS.jpg  
  inflating: dataset/test/Fake/017EMQKL4D.jpg  
  inflating: dataset/test/Fake/01877XHF3A.jpg  
  inflating: dataset/test/Fake/01EA4QAE1L.jpg  
  inflating: dataset/test/Fake/01FCG0FF23.jpg  
  inflating: dataset/test/Fake/01IUWPPCNT.jpg  
  inflating: dataset/test/Fake/01KDGEL6IQ.jpg  
  inflating: dataset/test/Fake/01MI46B2OH.jpg  
  inflati

In [16]:
!pip install torch torchvision

In [19]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_dataset = datasets.ImageFolder(root='dataset/train', transform=transform)

batch_size = 256
data_loader = DataLoader(image_dataset, batch_size=batch_size, shuffle=True)


image, label = image_dataset[0]
print("Example Image Shape:", image.shape)
print("Example Label:", label)

Example Image Shape: torch.Size([3, 224, 224])
Example Label: 0


In [22]:
from torchvision import datasets, transforms

# Define transforms for training and validation.
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Adjust these paths to match your loaded dataset
train_dir = 'dataset/train'  # update with your training folder path
val_dir   = 'dataset/valid'  # update with your validation folder path

# Create the datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transforms)

# Print the classes to verify correct loading
print("Classes found:", train_dataset.classes)


Classes found: ['Fake', 'Real']


In [23]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")


Number of training samples: 240002
Number of validation samples: 59428


In [24]:
import torch
import torch.nn as nn
from torchvision import models

# Load pretrained ResNet50
model = models.resnet50(pretrained=True)

# Freeze all layers so that we only train the final layer initially
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer to output a single value (for binary classification)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(model)


C:\Users\rutup\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rutup\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\rutup/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 33.8MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
import torch.optim as optim

# BCEWithLogitsLoss combines a sigmoid layer with binary cross entropy loss.
criterion = nn.BCEWithLogitsLoss()

# Only the final layer's parameters are being optimized initially.
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)


In [28]:
import time, copy

def train_model(model, criterion, optimizer, dataloaders, num_epochs=10, device='cuda'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-'*10)

        # Each epoch has a training and a validation phase.
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                loader = dataloaders['train']
            else:
                model.eval()
                loader = dataloaders['val']

            running_loss = 0.0
            running_corrects = 0
            total = 0

            # Iterate over the data.
            for inputs, labels in loader:
                inputs = inputs.to(device)
                labels = labels.to(device).float().unsqueeze(1)  # shape: (batch_size, 1)

                optimizer.zero_grad()

                # forward pass
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    preds = (torch.sigmoid(outputs) > 0.5).float()

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total += inputs.size(0)

            epoch_loss = running_loss / total
            epoch_acc = running_corrects.double() / total

            if phase=='train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc.item())

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model weights based on validation loss.
            if phase=='val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val loss: {best_loss:.4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)

    history = {'train_loss': train_losses, 'val_loss': val_losses,
               'train_acc': train_accs, 'val_acc': val_accs}
    return model, history

# Combine dataloaders into a dictionary.
dataloaders = {'train': train_loader, 'val': val_loader}


In [29]:
num_epochs = 5  # You can adjust this number as needed.
model, history = train_model(model, criterion, optimizer, dataloaders,
                             num_epochs=num_epochs, device=device)


Epoch 1/5
----------


KeyboardInterrupt: 

In [ ]:
# Uncomment below to fine-tune the last block of ResNet50.

for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Use a lower learning rate for fine-tuning.
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
num_epochs_ft = 5
model, history_ft = train_model(model, criterion, optimizer_ft, dataloaders,
                                num_epochs=num_epochs_ft, device=device)

# (Optional) Merge the fine-tuning history with the initial training history.
history['train_loss'].extend(history_ft['train_loss'])
history['val_loss'].extend(history_ft['val_loss'])
history['train_acc'].extend(history_ft['train_acc'])
history['val_acc'].extend(history_ft['val_acc'])


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        preds = (torch.sigmoid(outputs) > 0.5).int()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds).flatten()
all_labels = np.array(all_labels).flatten()

cm = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=train_dataset.classes)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)


In [ ]:
import seaborn as sns

plt.figure(figsize=(4,3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=train_dataset.classes,
            yticklabels=train_dataset.classes)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
def imshow(inp, title=None):
    """Display image for a Tensor."""
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.axis('off')

# Get one batch from the validation loader
inputs, labels = next(iter(val_loader))
inputs = inputs.to(device)
outputs = model(inputs)
preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy().flatten()
labels = labels.numpy()

plt.figure(figsize=(10,10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    pred_label = train_dataset.classes[preds[i]]
    true_label = train_dataset.classes[labels[i]]
    color = "green" if pred_label == true_label else "red"
    imshow(inputs[i], title=f"Pred: {pred_label}\nTrue: {true_label}")
    plt.title(f"Pred: {pred_label}\nTrue: {true_label}", color=color, fontsize=10)
plt.tight_layout()
plt.show()
